# Парсим хабр, чтобы собрать названия статей, ссылки на них и метки 

In [115]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm
from pathlib import Path

import pandas as pd

In [54]:
%%time
r = requests.get('https://habr.com/ru/articles/346198/')

C:\Users\trybi\anaconda3\envs\IndexHockey\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


Wall time: 463 ms


In [55]:
soup = BeautifulSoup(r.text)

In [56]:
soup.find('h1').text

'Как я парсил Хабр, часть 1: тренды'

In [62]:
[i.find('span').text for i in soup.find('div', {'class': 'tm-article-snippet__hubs'}).find_all('span', {'class': 'tm-article-snippet__hubs-item'})]

['Python',
 'Визуализация данных',
 'Исследования и прогнозы в IT',
 'Веб-аналитика']

In [78]:
n_to_pars = 15_000 # Вроде как не на каждый pid есть статья, так что возьмем с запасом
last_pid = 729916
pids = range(last_pid, last_pid-n_to_pars, -1)

results = []
for pid in tqdm(pids):
    url = f'https://habr.com/ru/articles/{pid}/'
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text)
        title = soup.find('h1').text
        tags = [i.find('span').text for i in (soup
                                              .find('div', {'class': 'tm-article-snippet__hubs'})
                                              .find_all('span',{'class': 'tm-article-snippet__hubs-item'}))]

        tags = ', '.join(tags)
        results.append([f'h{pid}', url, title, tags])

 52%|███████████████████████████████████████▋                                     | 7726/15000 [59:34<56:05,  2.16it/s]


AttributeError: 'NoneType' object has no attribute 'text'

In [85]:
df_results = pd.DataFrame(results, columns=['id', 'url', 'title', 'tags'])

In [87]:
already_parsed = set(df_results['id'])

In [108]:
results = []
for pid in tqdm(pids):
    if f'h{pid}' not in already_parsed:
        url = f'https://habr.com/ru/articles/{pid}/'
        r = requests.get(url)
        if r.status_code == 200:
            try:
                soup = BeautifulSoup(r.text)
                title = soup.find('h1').text
                tags = [i.find('span').text for i in (soup
                                                      .find('div', {'class': 'tm-article-snippet__hubs'})
                                                      .find_all('span',{'class': 'tm-article-snippet__hubs-item'}))]

                tags = ', '.join(tags)
                results.append([f'h{pid}', url, title, tags])
            except AttributeError:
                pass

  0%|                                                                                        | 0/15000 [00:00<?, ?it/s]C:\Users\trybi\anaconda3\envs\IndexHockey\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
100%|██████████████████████████████████████████████████████████████████████████| 15000/15000 [1:09:06<00:00,  3.62it/s]


In [109]:
results

[['h729662',
  'https://habr.com/ru/articles/729662/',
  'Создай своего ИИ-ассистента с помощью ChatGPT и Streamlit',
  'Блог компании Open Data Science, Python, API, Искусственный интеллект, Будущее здесь'],
 ['h723822',
  'https://habr.com/ru/articles/723822/',
  'Принцип Определения Дальности Между UWB Трансиверами (Конечный Автомат для DS-TWR)',
  'Алгоритмы, Беспроводные технологии, Программирование микроконтроллеров, Производство и разработка электроники'],
 ['h722188',
  'https://habr.com/ru/articles/722188/',
  'Американский суд в иске пользователя против Youtube принял сторону платформы и обязал истца оплатить издержек на $40 тыс',
  'Управление продажами, Облачные сервисы, Социальные сети и сообщества, Финансы в IT, IT-компании'],
 ['h722186',
  'https://habr.com/ru/articles/722186/',
  'Некоторые банки Кубы начали принимать карты «Мир»',
  'Платежные системы, Финансы в IT'],
 ['h722184',
  'https://habr.com/ru/articles/722184/',
  'Регулятор нагрузки',
  'C++, Разработка под

In [111]:
df_results_new = pd.DataFrame(results, columns=['id', 'url', 'title', 'tags'])

In [112]:
df_results_full = pd.concat([df_results, df_results_new])

In [113]:
df_results_full

,id,url,title,tags
0,h729916,https://habr.com/ru/articles/729916/,3D анимация Флекса или простого танца через ци...,"Разработка игр, Компьютерная анимация, Дизайн игр"
1,h729914,https://habr.com/ru/articles/729914/,Расширяя границы: компания «Тантор Лабс» запус...,PostgreSQL
2,h729912,https://habr.com/ru/articles/729912/,МТС в рамках допсоглашения урегулировала задол...,"IT-инфраструктура, Управление продажами, Финан..."
3,h729910,https://habr.com/ru/articles/729910/,Кто все вот эти на полках: краткое руководство...,"Блог компании Сбер, Гаджеты, Ноутбуки"
4,h729904,https://habr.com/ru/articles/729904/,Information: Microsoft работает над ИИ-чипами,"Машинное обучение, Производство и разработка э..."
...,...,...,...,...
3305,h714926,https://habr.com/ru/articles/714926/,"Про Vim "" Клиент БД",VIM
3306,h714924,https://habr.com/ru/articles/714924/,Специалисты Microsoft обвинили связанных с Ира...,"Информационная безопасность, IT-компании"
3307,h714922,https://habr.com/ru/articles/714922/,Команда Google Chromium разрабатывает браузер ...,"Разработка под iOS, Браузеры"
3308,h714920,https://habr.com/ru/articles/714920/,Общие шаблоны и нюансы использования React Query,"JavaScript, ReactJS"


In [116]:
data_path = Path('../data')

In [117]:
df_results_full.to_parquet(data_path / 'habr_posts.parquet')